In [1]:
import numpy as np
import pennylane as qml

from vqe_utils import *

# Variational Quantum Eigensolver (VQE)

In this example the number of qubits $n$ will be 2.

In [2]:
with open("100-2.in") as f:
    in2 = f.read()
# provided by QHack
H = parse_hamiltonian_input(in2)

In [3]:
num_qubits = len(H.wires)
print(num_qubits)

2


The computational basis is $|\phi_0>=|00>$, $|\phi_1>=10>$, etc. This gives us $N=2^n$ computational basis states.

Any other states of interest will be linear combinations (superpositions) of these basis states:
$$ \psi = \sum_i^N c_{i} \phi_i $$
where $c_{i}$ are complex numbers (with the restriction $\sum_i c_i^* c_i = 1$).

$H$ is a matrix $N\times N$ that can describes the interactions between the states of the computational basis. $H$ can be decomposed into products of the Pauli operators (e.g. $X_0X_1$, $X_0Z_2$, $Z_0Y_1X_2$, etc) and the expectation of these operators can be calculated on quantum computers.

In [65]:
print(in2)

+ 15.531709 I S + 0.218291 Z0 S - 6.125 Z1 S - 2.143304 X0 X1 S - 2.143304 Y0 Y1 S - 9.625 Z2 S - 3.913119 X1 X2 S - 3.913119 Y1 Y2


The eigenstates of $H$ are $|\Psi_0>$, ...
where $H|\Psi_i>=E_i|\Psi_i>$, with $E_i$ the energies of the system.

Eigenstates can be expressed as linear combinations of basis states.
$$ |\psi_j> = \sum_i^N c_{ij} |\phi_i> $$

Equivalently, a basis state can be transformed (rotated) into an arbitrary state by a unitary transformation $U(\theta)$. $$\psi(\theta) := U(\theta)\phi_0$$
$\theta$ is an alternate set of parameters ($c_{ij}=c_{ij}(\theta)$)

We can find the ground-state $\Psi_0$ by finding $\theta=\theta_0$ such that $\Psi(\theta_0)=\Psi_0$.

This means minimizing $E(\theta)=<\phi_0|U^\dagger(\theta)HU(\theta)|\phi_0>$, so we can treat $E(\theta)$ as the cost of an optimization problem.

The ground state energy is the "optimal value"
$$E_0 = \min_\theta <\phi_0|U^\dagger(\theta)HU(\theta)|\phi_0>$$

## VQE-100: Given $H$ and $U(\theta)$ find $\theta_{gs}$ corresponding to the ground state of $H$

Here we are given an ansatz. Given the normalization condition of $c$ our circuit only need $2^n-1$ complex parameters. The given ansatz has $(2^n-1)\times3$ real angles which should be enough.

In [4]:
num_param_sets = (2 ** num_qubits) - 1
params = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(num_param_sets, 3))

In [5]:
def variational_ansatz(params, wires):
    """
    DO NOT MODIFY anything in this function! It is used to judge your solution.

    This is ansatz is used to help with the problem structure. It applies
    alternating layers of rotations and CNOTs.

    Don't worry about the contents of this function for now—you'll be designing
    your own ansatze in a later problem.

    Args:
        params (np.ndarray): An array of floating-point numbers with size (n, 3),
            where n is the number of parameter sets required (this is determined by
            the problem Hamiltonian).
        wires (qml.Wires): The device wires this circuit will run on.
    """
    n_qubits = len(wires)
    n_rotations = len(params)

    if n_rotations > 1:
        n_layers = n_rotations // n_qubits
        n_extra_rots = n_rotations - n_layers * n_qubits

        # Alternating layers of unitary rotations on every qubit followed by a
        # ring cascade of CNOTs.
        for layer_idx in range(n_layers):
            layer_params = params[layer_idx * n_qubits : layer_idx * n_qubits + n_qubits, :]
            qml.broadcast(qml.Rot, wires, pattern="single", parameters=layer_params)
            qml.broadcast(qml.CNOT, wires, pattern="ring")

        # There may be "extra" parameter sets required for which it's not necessarily
        # to perform another full alternating cycle. Apply these to the qubits as needed.
        extra_params = params[-n_extra_rots:, :]
        extra_wires = wires[: n_qubits - 1 - n_extra_rots : -1]
        qml.broadcast(qml.Rot, extra_wires, pattern="single", parameters=extra_params)
    else:
        # For 1-qubit case, just a single rotation to the qubit
        qml.Rot(*params[0], wires=wires[0])

To execute the circuit we need a `quantum device`

In [22]:
dev = qml.device("default.qubit", wires=num_qubits)

Define the cost function with `ExpvalCost`

In [23]:
cost = qml.ExpvalCost(variational_ansatz, H, dev)

Initial energy:

In [24]:
print(cost(params))

17.545361261554945


What the circuit looks like:

In [30]:
@qml.qnode(dev)
def circuit(params):
    variational_ansatz(params,dev.wires)
    return qml.expval(qml.Identity(0))
result = circuit(params)
print(circuit.draw())

 0: ──Rot(0.171, 1.44, -0.534)──╭C────────────────────────────┤ ⟨I⟩ 
 1: ──Rot(0.326, 1.14, -0.169)──╰X──Rot(1.42, -0.289, 0.189)──┤     



Combine into a function and iterate `opt.step(cost)` to update parameters

In [49]:
def run_vqe(H):
    """Runs the variational quantum eigensolver on the problem Hamiltonian using the
    variational ansatz specified above.

    Fill in the missing parts between the # QHACK # markers below to run the VQE.

    Args:
        H (qml.Hamiltonian): The input Hamiltonian

    Returns:
        The ground state energy of the Hamiltonian.
    """
    # Initialize parameters
    num_qubits = len(H.wires)
    num_param_sets = (2 ** num_qubits) - 1
    params = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(num_param_sets, 3))

    energy = 0

    # QHACK #

    # Create a quantum device, set up a cost funtion and optimizer, and run the VQE.
    # (We recommend ~500 iterations to ensure convergence for this problem,
    # or you can design your own convergence criteria)
    dev = qml.device("default.qubit", wires=num_qubits)

    cost = qml.ExpvalCost(variational_ansatz, H, dev)

    opt = qml.GradientDescentOptimizer(0.1)
    #opt = qml.AdamOptimizer()
    
    max_iter = 200
    
    for i in range(max_iter):
        if i % 10 == 0: print(f"step {i}, cost {cost(params)}")
        params = opt.step(cost, params)

    energy = cost(params)

    # QHACK #

    # Return the ground state energy
    return energy

In [50]:
with open("100-1.in") as f:
    in1 = f.read()
print(in1)

+ 5.906709445000001 I S - 4.286607049870561 X0 S - 6.343290554999999 Z0


In [51]:
# Turn input to Hamiltonian
H = parse_hamiltonian_input(in1)

# Send Hamiltonian through VQE routine and output the solution
ground_state_energy = run_vqe(H)
print(f"{ground_state_energy:.6f}")

step 0, cost -0.3871706330336444
step 10, cost -1.7488323590361365
step 20, cost -1.7491585225867037
step 30, cost -1.7491598707282083
step 40, cost -1.7491598762984193
step 50, cost -1.7491598763214347
step 60, cost -1.7491598763215288
step 70, cost -1.7491598763215293
step 80, cost -1.7491598763215284
step 90, cost -1.7491598763215284
step 100, cost -1.7491598763215293
step 110, cost -1.7491598763215284
step 120, cost -1.7491598763215293
step 130, cost -1.7491598763215288
step 140, cost -1.7491598763215288
step 150, cost -1.7491598763215288
step 160, cost -1.7491598763215288
step 170, cost -1.7491598763215288
step 180, cost -1.7491598763215288
step 190, cost -1.7491598763215288
-1.749160


In [52]:
with open("100-1.ans") as f:
    ans1 = f.read()
print(ans1)

-1.749160



In [53]:
with open("100-2.in") as f:
    in2 = f.read()
print(in2)

+ 32.7658547225 I S - 2.1433035249352805 X1 S - 2.1433035249352805 Z0 X1 S + 17.015854722500002 Z1 S + 3.913118960624632 X0 Z1 S - 23.359145277499998 Z0 Z1 S - 3.913118960624632 X0 S - 26.859145277499998 Z0


In [54]:
# Turn input to Hamiltonian
H = parse_hamiltonian_input(in2)

# Send Hamiltonian through VQE routine and output the solution
ground_state_energy = run_vqe(H)
print(f"{ground_state_energy:.6f}")

step 0, cost 14.515723300157157
step 10, cost -1.8807310468339722
step 20, cost -1.923375238634108
step 30, cost -1.934335630463977
step 40, cost -1.9376339977143857
step 50, cost -1.9397334999266533
step 60, cost -1.941847279592146
step 70, cost -1.9442313556780775
step 80, cost -1.9469811507563506
step 90, cost -1.9501638098612233
step 100, cost -1.9538401583595792
step 110, cost -1.9580665945024833
step 120, cost -1.9628910558394352
step 130, cost -1.9683450590952738
step 140, cost -1.9744320256319803
step 150, cost -1.9811125383547505
step 160, cost -1.9882888208934553
step 170, cost -1.995793345397697
step 180, cost -2.0033890260993346
step 190, cost -2.0107884846276107
-2.017694


In [55]:
with open("100-2.ans") as f:
    ans1 = f.read()
print(ans1)

-2.045671


## VQE-200: Designing an ansatz $U(\theta)$

Suppose we know that the eigenstate must have a particular form.

We might want to map the quantum states of our basis to a one-hot encoding.

$$ \left\vert \psi(\alpha) \right\rangle = \alpha_0 \left\vert 100\cdots0 \right\rangle + \alpha_1 \left\vert 010\cdots0 \right\rangle + \cdots + \alpha_{n-2} \left\vert 0\cdots010 \right\rangle + \alpha_{n-1} \left\vert 0\cdots001 \right\rangle $$

where $\alpha_i$ are *real* numbers.

The strategy for this circuit is to iteratively apply Xs and RYs. The X puts one qubit into state |1>. The RY is a real number because Y is entirely imaginary ($RY(\theta)=e^{i\theta Y}$).

Base case:

$X_0|00> = |10>$

$RY(\theta)_1|10> = c(\theta)|10> + s(\theta)|11>$

Then entangle (using the CNOT to swap qubit0 back to 0:

$CNOT(0,1)(c(\theta)|10> + s(\theta)|11>) = c(\theta)|10> + s(\theta)|01>$

For more qubits we use controlled rotations to rotate the $i$th qubit on the term with 1 in $i-1$ qubit and then use a CNOT on the opposite wires to unflip the $i-1$th qubit.

Iterate:

`
for i in range(2,num_qubits):
    CRY(i-1,1)
    CNOT(i,i-1)
`

Example:

$CRY(\theta',1,2)[c(\theta)|100> + s(\theta)|010>] = c(\theta)|100> + s(\theta)[ c(\theta') |010> + s(\theta') |011> ]$

$CNOT(2,1)[c(\theta)|100> + s(\theta)[ c(\theta') |010> + s(\theta') |011> ] = c(\theta)|100> + s(\theta)c(\theta') |010> + s(\theta)s(\theta') |001>$

In [66]:
def variational_ansatz(params, wires):
    """The variational ansatz circuit.

    Fill in the details of your ansatz between the # QHACK # comment markers. Your
    ansatz should produce an n-qubit state of the form

        a_0 |10...0> + a_1 |01..0> + ... + a_{n-2} |00...10> + a_{n-1} |00...01>

    where {a_i} are real-valued coefficients.

    Args:
         params (np.array): The variational parameters.
         wires (qml.Wires): The device wires that this circuit will run on.
    """

    # QHACK #
    n_qubits = len(wires)
    qml.PauliX(wires=0)
    qml.RY(params[0], wires=1)
    qml.CNOT(wires=[1,0])

    if n_qubits > 2:
        for i in range(2,n_qubits):
            qml.CRY(params[i-1], wires=[i-1,i])
            qml.CNOT(wires=[i,i-1])
    # QHACK #

In [70]:
def run_vqe(H):
    """Runs the variational quantum eigensolver on the problem Hamiltonian using the
    variational ansatz specified above.

    Fill in the missing parts between the # QHACK # markers below to run the VQE.

    Args:
        H (qml.Hamiltonian): The input Hamiltonian

    Returns:
        The ground state energy of the Hamiltonian.
    """
    energy = 0

    # QHACK #

    # Initialize the quantum device
    num_qubits = len(H.wires)
    dev = qml.device("default.qubit", wires=num_qubits)

    # Randomly choose initial parameters (how many do you need?)
    params = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(num_qubits-1))

    # Set up a cost function
    cost = qml.ExpvalCost(variational_ansatz, H, dev)

    # Set up an optimizer
    opt = qml.GradientDescentOptimizer(0.1)

    max_iter = 400
    # Run the VQE by iterating over many steps of the optimizer
    for i in range(max_iter):
        if i % 50 == 0: print(f"step {i}, cost {cost(params)}")
        params = opt.step(cost, params)

    energy = cost(params)

    # QHACK #

    # Return the ground state energy
    return energy

In [71]:
with open("200-1.in") as f:
    in1 = f.read()
print(in1)

# Turn input to Hamiltonian
H = parse_hamiltonian_input(in1)

# Send Hamiltonian through VQE routine and output the solution
ground_state_energy = run_vqe(H)
print(f"{ground_state_energy:.6f}")

with open("200-1.ans") as f:
    ans1 = f.read()
print(f"Answer: {ans1}")

+ 5.906709 I S + 0.218291 Z0 S - 6.125 Z1 S - 2.143304 X0 X1 S - 2.143304 Y0 Y1
step 0, cost -1.6357002888302858
step 50, cost -1.7491612220155868
step 100, cost -1.7491612220155868
step 150, cost -1.7491612220155868
step 200, cost -1.7491612220155868
step 250, cost -1.7491612220155868
step 300, cost -1.7491612220155868
step 350, cost -1.7491612220155868
-1.749161
Answer: -1.749160



In [73]:
with open("200-2.in") as f:
    in2 = f.read()
print(in2)

# Turn input to Hamiltonian
H = parse_hamiltonian_input(in2)

# Send Hamiltonian through VQE routine and output the solution
ground_state_energy = run_vqe(H)
print(f"{ground_state_energy:.6f}")

with open("200-2.ans") as f:
    ans2 = f.read()
print(f"Answer: {ans2}")

+ 15.531709 I S + 0.218291 Z0 S - 6.125 Z1 S - 2.143304 X0 X1 S - 2.143304 Y0 Y1 S - 9.625 Z2 S - 3.913119 X1 X2 S - 3.913119 Y1 Y2
step 0, cost -0.5270569809238184
step 50, cost -2.0456709937085686
step 100, cost -2.045672287677065
step 150, cost -2.04567228767709
step 200, cost -2.0456722876770908
step 250, cost -2.0456722876770908
step 300, cost -2.0456722876770908
step 350, cost -2.0456722876770908
-2.045672
Answer: -2.045671


## VQE-500: Find the excited states of $H$

VQE as described in the above seems very limited compared to classical diagonalization. We also want the excited states in order to fully describe real systems (e.g. molecules, nuclei).

### Attempt #1: Variational Quantum Deflation

One strategy is to iterate VQE but modify the cost function to force the lowest state to be orthogonal to all the previously found states.

### Attempt #2: Weighted Subspace-search VQE